In [4]:
import pandas as pd

user_data = pd.read_csv('userprofile.csv')

user_data = user_data.drop(['latitude', 'longitude'], axis = 1)
user_data['smoker'] = user_data['smoker'].apply(lambda x: 0 if x=='false' else 1)
user_data['birth_year'] = user_data['birth_year'].astype('str')

user_payment = pd.read_csv('userpayment.csv')
user_data = user_data.merge(user_payment, on='userID', how='inner')

user_cuisine = pd.read_csv('usercuisine.csv')
user_data = user_data.merge(user_cuisine, on='userID', how='inner')

cat_cols = ['drink_level', 'dress_preference', 'ambience', 'transport', 'marital_status', 'hijos', 'birth_year', 'interest', 'personality', 'religion', 'activity', 'color', 'budget', 'Upayment', 'Rcuisine']
user_data = pd.get_dummies(user_data, columns=cat_cols)

user_data = user_data.rename({'userID': 'user_idx'}, axis = 1)
user_data['user_idx'] = user_data['user_idx'].apply(lambda x: x.strip())

user_ids = user_data['user_idx'].unique()
user2ext = {id_ : k for k, id_ in enumerate(user_ids)}
user_data['user_idx'] = user_data['user_idx'].map(user2ext)

In [2]:
ratings = pd.read_csv('rating_final.csv')
ratings['relevance'] = (ratings['rating'] + ratings['food_rating'] + ratings['service_rating']) / 3
ratings = ratings.drop(['rating', 'food_rating', 'service_rating'], axis = 1)
ratings = ratings.rename({'userID': 'user_idx', 'placeID': 'item_idx'}, axis = 1)
ratings['item_idx'] = ratings['item_idx'].astype(str)

ratings['item_idx'] = ratings['item_idx'].apply(lambda x: x.strip())
ratings['user_idx'] = ratings['user_idx'].apply(lambda x: x.strip())
ratings['user_idx'] = ratings['user_idx'].map(user2ext)

In [3]:
item_data = pd.read_csv('geoplaces2.csv')
item_cuisine = pd.read_csv('chefmozcuisine.csv')
item_hours = pd.read_csv('chefmozhours4.csv')
item_parking = pd.read_csv('chefmozparking.csv')
item_accept = pd.read_csv('chefmozaccepts.csv')

item_data = item_data.merge(item_cuisine, on='placeID', how='inner')
item_data = item_data.merge(item_hours, on='placeID', how='inner')
item_data = item_data.merge(item_parking, on='placeID', how='inner')
item_data = item_data.merge(item_accept, on='placeID', how='inner')

item_data = item_data.drop(['latitude', 'longitude', 'the_geom_meter', 'name', 'address', 'city', 'state', 'country', 'fax', 'url', 'zip'], axis = 1)
cat_cols = ['alcohol', 'smoking_area', 'dress_code', 'accessibility', 'price', 'Rambience', 'franchise', 'area', 'other_services', 'Rcuisine', 'parking_lot', 'Rpayment']

item_data['open_hrs'] = item_data['hours'].apply(lambda x: int(x.split('-')[0].split(':')[0]))
item_data['close_hrs'] = item_data['hours'].apply(lambda x: int(x.split('-')[0].split(':')[0]))
item_data = item_data.drop(['days', 'hours'], axis = 1)
item_data = item_data.rename({'placeID': 'item_idx'}, axis = 1)
item_data = pd.get_dummies(item_data, columns=cat_cols)
item_data['item_idx'] = item_data['item_idx'].astype(str)
item_data['item_idx'] = item_data['item_idx'].apply(lambda x: x.strip())

In [4]:
item_data.to_parquet("preprocessed/items.parquet", index=False)
user_data.to_parquet("preprocessed/users.parquet", index=False)
ratings.to_parquet("preprocessed/rating.parquet", index=False)